In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv(open('countix/countix_val.csv'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1433 entries, 0 to 1432
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   video_id          1433 non-null   object 
 1   class             1433 non-null   object 
 2   kinetics_start    1433 non-null   int64  
 3   kinetics_end      1433 non-null   int64  
 4   repetition_start  1433 non-null   float64
 5   repetition_end    1433 non-null   float64
 6   count             1433 non-null   int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 78.5+ KB


In [9]:
def df2link(df):
    return 'https://youtube.com/watch?v=' + df['video_id'] + '&t=' + \
        df['kinetics_start'].astype(str) + 's'

In [10]:
from IPython.display import HTML

cate = df[df['class'] == 'situp'].sample(n=1)
x = df2link(cate).item()
HTML(f"""<a href={x}>{x}</a>""")

In [11]:
actions = ['exercising arm', 'bench pressing', 'front raises', 'squat', 'jumping jacks',
     'lunge', 'mountain climber (exercise)', 'pull ups', 'push up', 'situp']

df[df['class'].isin(actions)].to_csv('workouts_val.csv')

In [12]:
df = pd.read_csv('workouts_val.csv', index_col=0)
df.index.name = 'index'
df.head()

,video_id,class,kinetics_start,kinetics_end,repetition_start,repetition_end,count
index,,,,,,,
47,eovgxTEV-sE,jumping jacks,0,10,0.000000,10.000000,2
49,QFcCRwncPmM,bench pressing,0,10,4.960000,9.560000,2
50,QiJ1swCeKQQ,bench pressing,46,56,46.806072,53.724861,2
51,QkGng8veoS8,bench pressing,1,11,1.066733,9.241575,2
52,Q5YcyqFRvRM,bench pressing,97,107,97.033367,104.941275,2


## Download

In [13]:
%pip install yt-dlp -q
import yt_dlp

Note: you may need to restart the kernel to use updated packages.


In [ ]:
def download_ytb(vid, folder):
    url = 'https://www.youtube.com/watch?v='
    link = url + vid
    ydl_opts = {
        'outtmpl': f'{folder}/{vid}',
        'quiet': True,
        'ignoreerrors': True,
        'format': 136, # 136 for mp4 1280x720 30fps no audio 
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([link])

# TODO use multi-thread
df.apply(lambda row: download_ytb(row['video_id'], 'downloads'), axis=1)

## Process videos

In [ ]:
import os
import subprocess

vids = os.listdir('/content/ALL')
dest = '/content/clips/'
print(len(vids))
def cut():
    for vid in vids:
        print(vid)
        v = df[df['video_id'] == vid.split('.')[0]].iloc[0]
        if not len(v): # .DS_store
            continue
        name = v['video_id']
        ss = v['kinetics_start']
        to = v['kinetics_end']
        infile = os.path.join('/content/ALL/', vid)
        outfile = os.path.join(dest, name+'.mp4')
        cmd = f"ffmpeg -i {infile} -ss {ss} -t 10 -an {outfile}"
        print(cmd)
        subprocess.run([cmd], shell=True, capture_output=True)

In [ ]:
import base64

def download_onedrive(link):
    b = base64.urlsafe_b64encode(link.strip().encode('ascii'))
    s = b.decode('ascii') # seems that 'Qnc=' is fine
    res = f'https://api.onedrive.com/v1.0/shares/u!{s}/root/content'
    return res

link = 'https://1drv.ms/u/s!'
download_onedrive(link)